In [15]:
using LinearAlgebra
using BenchmarkTools
using LaTeXStrings
using QuantumOptics
using CairoMakie
using BenchmarkTools
using QuantumFCS

In [19]:
N = 100
b_tls = NLevelBasis(3)
b_fock = FockBasis(N)
σ = [transition(b_tls, i, j) for i=1:3, j =1:3];
a = destroy(b_fock)
ad = create(b_fock)
Id_cav = one(b_fock)
Id_tls = one(b_tls)
;

In [20]:
g = 1.0 
f = 0.1
Δ = 0.0
γh = 1.0
γc = 1.0
κ = 1.0
nh = 1.0
nc = 0.1
ncav = 0.0


H = g*ad⊗σ[1,2] + f*a⊗one(b_tls)
H += dagger(H)

rates = [γh*√(nh+1), γc*√(nc+1), κ*√(ncav+1), γh*√nh, γc*√nc, κ*√ncav]
J = [Id_cav⊗σ[1,3], Id_cav⊗σ[2,3], a⊗Id_tls]
for i=1:3
    append!(J,[dagger(J[i])])
end

# J = @. rates*J


ρss = steadystate.iterative(H, J; rates=rates)
# ρss = (ρss+dagger(ρss))/2
# tout, ρss = steadystate.master(H, J; rates=rates, Jdagger=dagger.(J));
;

In [18]:
mJ=[J[2], dagger(J[2])]
@time fcscumulants_recursive(H, J, mJ, 2, ρss)

  0.076345 seconds (1.28 k allocations: 278.143 MiB, 0.00% compilation time)


2-element Vector{Float64}:
 0.014919888329598608
 0.15496976511514832

In [21]:
mJ=[J[2], dagger(J[2])]
@time fcscumulants_recursive(H, J, mJ, 2, ρss)

  5.816497 seconds (1.31 k allocations: 8.455 GiB, 12.96% gc time, 0.00% compilation time)


2-element Vector{Float64}:
 0.014919886082694128
 0.1549697554668058

In [8]:
L=steadystate.liouvillian(H, J; rates=rates, Jdagger=dagger.(J))

SparseSuperOpType{Tuple{CompositeBasis{Vector{Int64}, Tuple{FockBasis{Int64}, NLevelBasis{Int64}}}, CompositeBasis{Vector{Int64}, Tuple{FockBasis{Int64}, NLevelBasis{Int64}}}}, Tuple{CompositeBasis{Vector{Int64}, Tuple{FockBasis{Int64}, NLevelBasis{Int64}}}, CompositeBasis{Vector{Int64}, Tuple{FockBasis{Int64}, NLevelBasis{Int64}}}}, SparseArrays.SparseMatrixCSC{ComplexF64, Int64}}(([Fock(cutoff=10) ⊗ NLevel(N=3)], [Fock(cutoff=10) ⊗ NLevel(N=3)]), ([Fock(cutoff=10) ⊗ NLevel(N=3)], [Fock(cutoff=10) ⊗ NLevel(N=3)]), sparse([1, 2, 34, 749, 1, 2, 3, 12, 35, 750  …  1055, 1087, 1088, 1089, 341, 715, 1055, 1056, 1088, 1089], [1, 1, 1, 1, 2, 2, 2, 2, 2, 2  …  1088, 1088, 1088, 1088, 1089, 1089, 1089, 1089, 1089, 1089], ComplexF64[-1.0 + 0.0im, 0.0 - 1.0im, 0.0 + 1.0im, 1.0 + 0.0im, 0.0 - 1.0im, -1.5 + 0.0im, 0.0 - 1.4142135623730951im, 0.0 - 1.0im, 0.0 + 1.0im, 1.0 + 0.0im  …  0.0 + 3.1622776601683795im, 0.0 - 3.0im, -11.963022410543246 + 0.0im, 0.0 - 3.1622776601683795im, 1.4142135623730951

In [40]:
ρss = steadystate.eigenvector(L)

Operator(dim=33x33)
  basis: [Fock(cutoff=10) ⊗ NLevel(N=3)]
    0.0305305-2.6e-16im         6.1e-16+0.0209321im   …   1.0e-17+1.0e-17im
      6.4e-16-0.0209321im     0.0416564-0.0im             1.0e-17-0.0im
   0.00446386-7.3e-16im         9.4e-16-0.0390006im          -0.0-0.0im
     -4.8e-16-0.0255414im    -0.0165147-1.36e-15im            0.0+0.0im
   -0.0322347-3.0e-17im       -1.38e-15-0.00920365im         -0.0-0.0im
     -3.7e-16+0.0266596im     -0.024732+7.5e-16im     …  -1.0e-17-0.0im
    0.0161225+5.2e-16im         1.3e-16+0.0272066im      -1.0e-17+1.0e-17im
      5.1e-16-0.00665674im    0.0211879+2.4e-16im             0.0+0.0im
 -0.000816622-4.1e-16im         2.6e-16-0.0127571im      -1.0e-17+1.0e-17im
     -1.5e-16-0.00167496im  -0.00571423-1.9e-16im            -0.0-0.0im
             ⋮                                        ⋱  
     -1.0e-17+0.0im                -0.0-1.0e-17im         3.0e-17-2.0e-17im
         -0.0-0.0im            -1.0e-17-1.0e-17im     …      -0.0-3.0e-17

In [51]:
isapprox(ρss,dagger(ρss))

true